In [ ]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.stem.porter import *
import string
import re
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, ConfusionMatrixDisplay
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
import gc
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
stopwords = stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text


def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens


def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]+", tweet.lower())).strip()
    return tweet.split()


vectorizer_david = TfidfVectorizer(tokenizer=tokenize,
                                   preprocessor=preprocess,
                                   ngram_range=(1, 3),
                                   stop_words=stopwords,
                                   use_idf=True,
                                   smooth_idf=False,
                                   norm=None,
                                   decode_error='replace',
                                   max_features=10000,
                                   min_df=5,
                                   max_df=0.75)

pos_vectorizer_david = TfidfVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None,
    use_idf=False,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.75,
)

sentiment_analyzer = VS()


def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return (parsed_text.count('URLHERE'), parsed_text.count('MENTIONHERE'),
            parsed_text.count('HASHTAGHERE'))


def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)

    words = preprocess(tweet)  #Get text only

    syllables = textstat.syllable_count(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables + 0.001)) / float(num_words + 0.001), 4)
    num_unique_terms = len(set(words.split()))

    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(
        float(0.39 * float(num_words) / 1.0) + float(11.8 * avg_syl) - 15.59,
        1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(
        206.835 - 1.015 * (float(num_words) / 1.0) - (84.6 * float(avg_syl)),
        2)

    twitter_objs = count_twitter_objs(tweet)
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [
        FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms,
        num_words, num_unique_terms, sentiment['neg'], sentiment['pos'],
        sentiment['neu'], sentiment['compound'], twitter_objs[2],
        twitter_objs[1], twitter_objs[0], retweet
    ]
    #features = pandas.DataFrame(features)
    return features


def get_feature_array(tweets):
    feats = []
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()

sentiment_analyzer = VS()

pos_vectorizer = TfidfVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None,
    use_idf=False,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.75,
)

vectorizer = TfidfVectorizer(tokenizer=basic_tokenize,
                             preprocessor=preprocess,
                             ngram_range=(1, 3),
                             stop_words=stopwords,
                             use_idf=True,
                             smooth_idf=False,
                             norm=None,
                             decode_error='replace',
                             max_features=10000,
                             min_df=5,
                             max_df=0.75)

davidson_model = LogisticRegression(class_weight='balanced', penalty='l2')

hate_map = {0: "NON HATE", 1: "HATE"}

In [ ]:
def get_vectorisers(train_tweets):
    global vectorizer_david, pos_vectorizer_david
    vectorizer_david = vectorizer_david.fit(train_tweets)
    train_tweet_tags = get_tag_list(train_tweets)
    pos_vectorizer_david = pos_vectorizer_david.fit(
        pd.Series(train_tweet_tags))


def reset_vectorisers():
    global vectorizer_david, pos_vectorizer_david
    vectorizer_david = TfidfVectorizer(tokenizer=tokenize,
                                       preprocessor=preprocess,
                                       ngram_range=(1, 3),
                                       stop_words=stopwords,
                                       use_idf=True,
                                       smooth_idf=False,
                                       norm=None,
                                       decode_error='replace',
                                       max_features=10000,
                                       min_df=5,
                                       max_df=0.75)

    pos_vectorizer_david = TfidfVectorizer(
        tokenizer=None,
        lowercase=False,
        preprocessor=None,
        ngram_range=(1, 3),
        stop_words=None,
        use_idf=False,
        smooth_idf=False,
        norm=None,
        decode_error='replace',
        max_features=5000,
        min_df=5,
        max_df=0.75,
    )


def get_tag_list(tweets):
    tweet_tags = []
    for tweet in tweets:
        tokens = basic_tokenize(preprocess(tweet))
        tags = nltk.pos_tag(tokens)
        tag_list = [x[1] for x in tags]
        tag_str = " ".join(tag_list)
        tweet_tags.append(tag_str)
    return tweet_tags


def return_feature_set(tweets):
    global vectorizer_david, pos_vectorizer_david
    tfidf = vectorizer_david.transform(tweets).toarray()
    tweet_tags = get_tag_list(tweets)
    pos = pos_vectorizer_david.transform(pd.Series(tweet_tags)).toarray()
    feats = get_feature_array(tweets)
    feat_M = np.concatenate([tfidf, pos, feats], axis=1)
    del tfidf, pos, feats, tweet_tags
    gc.collect()
    return feat_M


def run_model(train_texts, train_labels):
    reset_vectorisers()
    get_vectorisers(train_texts)
    X_train = return_feature_set(train_texts)
    y_train = np.asarray(train_labels)

    base_model = LogisticRegression()
    base_model.fit(X_train, y_train)
    print("TRAIN ACCURACY")
    y_preds = base_model.predict(X_train)
    report = classification_report(y_train, y_preds)
    print(report)
    return base_model

In [ ]:
folder = "ext_eval"
david = pd.read_csv(os.path.join(folder, "davidson_raw_labeled_data.csv"))
david['class'].hist()
plt.show()
print(david.groupby('class').count())

train_text = david['tweet']
train_label = david['class']
train_label_final = []
for i in train_label:
    if i == 1 or i == 0:
        train_label_final.append(1)  # hate is 1
    else:
        train_label_final.append(0)
plt.hist(train_label_final)
plt.show()
c = list(zip(train_text, train_label_final))
random.shuffle(c)
train_texts, train_labels = zip(*c)

train_texts = list(train_texts)
train_labels = list(train_labels)

model = run_model(train_texts, train_labels)

## Results

In [ ]:
def ext_eval_probs(yg, yp):
    diff = []
    for g, p in zip(yg, yp):
        if g >= 0.5 and p >= 0.5:  ## 0 is hate label so we take <0.5
            diff.append(g - p)
    return np.mean(diff)


def run_for_test(model_name):
    folder = "ext_eval"
    print(model_name)
    file = os.path.join(folder, model_name + "_for_ext_eval.pkl")
    with open(file, "rb") as f:
        data = pickle.load(f)
    xg = data["ground"]
    xp = data["pred"]
    xg = return_feature_set(xg)
    xp = return_feature_set(xp)
    yg = model.predict_proba(xg)[:, 1]
    yp = model.predict_proba(xp)[:, 1]
    print(ext_eval_probs(yg, yp))


def run_for_test_dict(model_name):
    folder = "ext_eval"
    print(model_name)
    file = os.path.join(folder, model_name + "_for_ext_eval.pkl")
    with open(file, "rb") as f:
        data = pickle.load(f)
    for k in data:
        print("------k-----", k)
        xg = data[k]["ground"]
        xp = data[k]["pred"]
        xg = return_feature_set(xg)
        xp = return_feature_set(xp)
        yg = model.predict_proba(xg)[:, 1]
        yp = model.predict_proba(xp)[:, 1]
        print(ext_eval_probs(yg, yp))


def execute_():
    run_for_test("neutral")
    run_for_test("drgpreds")
    run_for_test("ntpcares")
    run_for_test_dict("fgst")
    run_for_test_dict("style")
    run_for_test("nacl")

In [ ]:
execute_()